In [3]:
import os
os.getcwd()

'C:\\Users\\2-Prog'

In [1]:
#Shopping App
import uuid as uid

class ShoppingApp:

    def __init__(self):
        self.loggedUsers = []
        self.categories = []
        self.catalogs = [] 
        self.cart = {}

        #Display the Banner
        self.banner()
        self.userInfos = UserInfo()
        self.populateCatalog()

    # Function to display Welcome Banner
    def banner(self):
        print('#'*33)
        print(r'## Welcome to Demo Marketplace ##')
        print('#'*33)

    # Login functionality. User is authenticated if username and password
    # entered matches with the configured username and password
    def login(self, userId, password):
        for user in self.userInfos.users:
            if( userId == user['id'] and password == user['password']):
                    sessId = uid.uuid4()
                    print(f"User: {user['name']} successfully logged in as admin")
                    self.loggedUsers.append({"sessionId": sessId, "userId": user['id'], "isAdmin": user['is_admin']})
                    return sessId;

    # function to populate the product catalogue
    def populateCatalog(self): 
        
        #populate categories
        self.categories.append({"id": "c001", "name": "Boots"})
        self.categories.append({"id": "c002", "name": "Coats"})
        self.categories.append({"id": "c003", "name": "Jackets"})
        self.categories.append({"id": "c004", "name": "Caps"})

        #populate products with categories
        self.catalogs.append({"id": "p1001", "name": "RedTape", "cat_id": "c001", "price": 1001.99})
        self.catalogs.append({"id": "p1002", "name": "Canvas", "cat_id": "c001", "price": 250.49})
        self.catalogs.append({"id": "p1003", "name": "Bata", "cat_id": "c001", "price": 99.99})
        self.catalogs.append({"id": "p1004", "name": "Nike", "cat_id": "c001", "price": 4021.99})

        self.catalogs.append({"id": "p2001", "name": "Raymond", "cat_id": "c002", "price": 799.99})
        self.catalogs.append({"id": "p2002", "name": "VanHussein", "cat_id": "c002", "price": 999.99})
        self.catalogs.append({"id": "p2003", "name": "Peter England", "cat_id": "c002", "price": 899.99})

        self.catalogs.append({"id": "p3001", "name": "Urban", "cat_id": "c003", "price": 499.49})
        self.catalogs.append({"id": "p3002", "name": "Adidas", "cat_id": "c003", "price": 589.49})
        self.catalogs.append({"id": "p3003", "name": "Spyder", "cat_id": "c003", "price": 699.49})

        self.catalogs.append({"id": "p4001", "name": "Flames", "cat_id": "c004", "price": 49.49})
        self.catalogs.append({"id": "p4002", "name": "RedFLag", "cat_id": "c004", "price": 19.49})
        self.catalogs.append({"id": "p4003", "name": "Vans", "cat_id": "c004", "price": 39.49})


    # function to update the catalogue. Only admin is allowed to perfomr this operation 
    def updateCatalog(self, sessionId):
        
        if not self.isLoggedInUserAdmin(sessionId):
            print('!!! Invalid action: Only Admins are allowed to update the Catalogue')
            return;
        
        self.categories.append({"id": "c005", "name": "Bombs"})
        self.catalogs.append({"id": "p5001", "name": "S-400", "cat_id": "c005", "price": 99999.99})
        self.catalogs.append({"id": "p5002", "name": "Brahmos", "cat_id": "c005", "price": 9999.99})
        self.catalogs.append({"id": "p5003", "name": "Akash", "cat_id": "c005", "price": 9999.99})

    # function to delete from the catalogue. Only admin is allowed to perform this operation
    def deleteFromCatalog(self, sessionId, productId):
        
        if not self.isLoggedInUserAdmin(sessionId):
            print('!!! Invalid action: Only Admins are allowed to remove items from the Catalogue')
            return;
            
        self.catalogs = [product for product in self.catalogs if product.get('id') != productId]
        

    # function to display the Catalogue 
    def displayCatalog(self, sessionId):

        if any(userSession['sessionId'] == sessionId for userSession in self.loggedUsers):
            # Print the Catalog
            print("printing catalog")
            print('#'*75)
            print( f"{'id':10} {'Name':37} {'Price':15} {'Catagory':10s}")
            print('-'*75)
            for catg in self.catalogs:
                print(f"{catg['id']:<10} {catg['name']:<20s} \t\t {catg['price']:<10} \t {self.getCategoryNameForId(catg['cat_id']):<10s}")               
            print('#'*75)
        else:
            print(' !!! Not Authorized to view the catalog ')

    # function to add item to cart. Only non-admin users can do this operation        
    def addToCart(self, sessionId, prodId, quantity=1):

        if self.isLoggedInUserAdmin(sessionId):
            print('!!! Invalid action: Admin not allowed to add item to Cart')
            return;
            
        productFound = False;
        
        if all( productCtgs['id'] != prodId for productCtgs in self.catalogs):
            print(" !!! invalid product Id :", prodId)
            return
        
        if sessionId in self.cart:
            cart = self.cart[sessionId]
            for prod in cart:
                if( prod['product_id'] == prodId):
                   # print(f'found {prod['product_id']} with price: {prod['price']} with name: {self.catalogs['id'==prodId]['name']}')
                    productFound = True
                    prod['quantity'] += quantity
                    prod['price'] = list(filter(lambda catalog: catalog['id']==prodId, self.catalogs))[0]['price'] * prod['quantity']
                    break;
            if not productFound:
                price = list(filter(lambda catalog: catalog['id']==prodId, self.catalogs))[0]['price']
                cart.append({'product_id': prodId, 'quantity': quantity, 'price': price})
        else:
            price = list(filter(lambda catalog: catalog['id']==prodId, self.catalogs))[0]['price']
            self.cart[sessionId] = [{'product_id': prodId, 'quantity': quantity, 'price': price}]

    # function to remove item from cart. Only non-admin users can do this operation
    def removeFromCart(self, sessionId, productId):
        
        if self.isLoggedInUserAdmin(sessionId):
            print('!!! Invalid action: Admin not allowed to add item to Cart')
            return;
            
        if sessionId in self.cart:
            cart = self.cart[sessionId]
            self.cart[sessionId] = [prodItem for prodItem in cart if prodItem.get('product_id') != productId]             
         
        else:
            print("!!! No Cart found for this user")
        
    # function to display the cart. Only non-admin users can do this operation.         
    def displayCart(self, sessionId):
        
        if self.isLoggedInUserAdmin(sessionId):
            print('!!! Invalid action: Admin not allowed to add item to Cart')
            return;
            
        if sessionId in self.cart:
            cart = self.cart[sessionId]
            sum = 0
            print(f"{'Product Name':25} {'Price':20} {'Quantity':20} {'Total':20}")
            print('-'*90)
            for prod in cart:
                price = list(filter(lambda catalog: catalog['id']==prod['product_id'], self.catalogs))[0]['price']
                print(f"Name:{self.getProductNameForId(prod['product_id']):15s} {price:10} {prod['quantity']:20} {prod['price']:20}")
                sum += prod['price']
            print('-'*90)
            print(f' Total : {sum}.'.rjust(75, ' '))
            print('-'*90)
              
        else:
            print(' !!! Empty Cart. Please shop with us.. Pleaseeee')

    # function to make a payment for the items in the cart. Only non-admin user can do this operation        
    def makePayment(self, sessionId, paymentMode='UPI'):
        if self.isLoggedInUserAdmin(sessionId):
            print('!!! Invalid action: Admin not allowed to make payment to the  Cart')
            return;
        if sessionId in self.cart:
            sum = 0
            cart = self.cart[sessionId]
            for prod in cart:
                price = list(filter(lambda catalog: catalog['id']==prod['product_id'], self.catalogs))[0]['price']
                sum += price
            print(f'Please make the payment of Rs:{sum} via {paymentMode}')
            print(f'$'*5 )
            print(f'Payment received for amount Rs:{sum}')
            print('Thank you for making the payment')
            del self.cart[sessionId]

        else:
            print(' !!! No items found in the cart ')

    # Helper Function - to retreive category name from category Id
    def getCategoryNameForId(self, catId):
        for catg in self.categories:
            if( catId == catg['id']):
                return catg['name']

    # Helper Function - to retreive product name from product Id
    def getProductNameForId(self, prodId):
        for prod in self.catalogs:
            if( prodId == prod['id']):
                return prod['name']
                
    # Helper Function - to check whether the logged in user is admin user based on the sessionId
    def isLoggedInUserAdmin(self, sessionId):
        return list(filter(lambda loggedUser: loggedUser['sessionId'] == sessionId, shopApp.loggedUsers))[0]['isAdmin']
        print("\n Catalogue:")
        

#################
# User Database #
#################

class UserInfo:
    
    def __init__(self):
        self.users = []
        print('\ncreating demo database for users and admin\n')
        self.populateUsers()

    # function to populate the users
    def populateUsers(self):
        self.users.append({"name": "Charles", "id": "charl_123", "password": "charl@123", "is_admin": False})
        self.users.append({"name": "Mike", "id": "mike_123", "password": "mike@123", "is_admin": False})
        self.users.append({"name": "Admin", "id": "admin", "password": "admin@123", "is_admin": True})

    # function to get user name from user id
    def getUserNameForId(self, userId):
        for user in self.users:
            if userId == user['id']:
                return user['name']

    # function to check whether the user has admin role
    def isUserAdmin(self, userId):
        for user in self.users:
            if userId == user['id']:
                return user['is_admin']

###########
## Start ##
###########

if __name__ == '__main__':
    shopApp = ShoppingApp()
    
    # User and Admin logins
    mikeSessionId = shopApp.login('mike_123', 'mike@123')
    adminSessionId = shopApp.login('admin', 'admin@123')

    #Displaying the Catalogues for mike, admin and invalid user
    print('\nDisplaying the catalog for user mike')
    shopApp.displayCatalog(mikeSessionId)
    print('\nDisplaying the catalog for user admin')
    shopApp.displayCatalog(adminSessionId)
    print('\nDisplaying the catalog for invalid User')
    shopApp.displayCatalog('invalid-sessionid')

    # Purchase done by Mike
    shopApp.addToCart(mikeSessionId, 'p1001', 1);
    shopApp.addToCart(mikeSessionId, 'p1002', 1);
    
    print('\n User Mike cart ')
    shopApp.displayCart(mikeSessionId)
    
    print('\n User Mike added one more item which was already in his cart')
    shopApp.addToCart(mikeSessionId, 'p1002', 1);
    shopApp.displayCart(mikeSessionId)

    print('\n User Mike removed the product p1002 from the cart')
    shopApp.removeFromCart(mikeSessionId, 'p1002')
    shopApp.displayCart(mikeSessionId)

    print('\n User Mike making payment ')
    shopApp.makePayment(mikeSessionId)

    print('\n Checking the role of the user')
    isMikeAdmin = list(filter(lambda loggedUser: loggedUser['userId'] == 'mike_123', shopApp.loggedUsers))[0]['isAdmin']
    isAdminAdmin = list(filter(lambda loggedUser: loggedUser['userId'] == 'admin', shopApp.loggedUsers))[0]['isAdmin']
    print(f'Can user mike act as admin:: {isMikeAdmin}')
    print(f'Can user admin act as admin:: {isAdminAdmin}')

    print('\n Admin user trying to add item to cart')
    shopApp.addToCart(adminSessionId, 'p1001', 1);
    print('\n Admin user trying to make payment to cart')
    shopApp.makePayment(adminSessionId);

    print('\n Displaying Catalog before update by admin')
    shopApp.displayCatalog(adminSessionId)
    shopApp.updateCatalog(adminSessionId)
    print('\n Displaying Catalog after update by admin')
    shopApp.displayCatalog(adminSessionId)


    print('\n Displaying Catalog before delete of item by admin')
    shopApp.displayCatalog(adminSessionId)
    shopApp.deleteFromCatalog(adminSessionId, 'p5002')
    print('\n Displaying Catalog after delete of item p5002 by admin')
    shopApp.displayCatalog(adminSessionId)

    print('\n User Mike trying to update the catalog')
    shopApp.updateCatalog(mikeSessionId)
    print('\n User Mike trying to delete item from the catalog')
    shopApp.deleteFromCatalog(mikeSessionId, 'p5002')
    
    


#################################
## Welcome to Demo Marketplace ##
#################################

creating demo database for users and admin

User: Mike successfully logged in as admin
User: Admin successfully logged in as admin

Displaying the catalog for user mike
printing catalog
###########################################################################
id         Name                                  Price           Catagory  
---------------------------------------------------------------------------
p1001      RedTape              		 1001.99    	 Boots     
p1002      Canvas               		 250.49     	 Boots     
p1003      Bata                 		 99.99      	 Boots     
p1004      Nike                 		 4021.99    	 Boots     
p2001      Raymond              		 799.99     	 Coats     
p2002      VanHussein           		 999.99     	 Coats     
p2003      Peter England        		 899.99     	 Coats     
p3001      Urban                		 499.49     	 Jackets   
p3002      